In [14]:
!pip install pygraphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pygraphviz (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [63 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-11.1-arm64-cpython-311
      creating build/lib.macosx-11.1-arm64-cpython-311/pygraphviz
      copying pygraphviz/scraper.py -> build/lib.macosx-11.1-arm64-cpython-311/pygraphviz
      copying pygraphviz/graphviz.py -> build/lib.macosx-11.1-arm64-cpython-311/pygraphviz
      copying pygraphviz/__init__.py -> build/lib.macosx-11.1-arm64-cpython-311/pygraphviz
      copying pygraphviz/agraph.py -> build/lib.macosx-11.1-arm64-cpython-311/pygraphviz
      c

In [2]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 4.3 MB/s eta 0:00:00


In [3]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [21]:
%set_env OPENAI_API_KEY=1
%set_env TAVILY_API_KEY=t2

env: OPENAI_API_KEY=1
env: TAVILY_API_KEY=t2


In [7]:
tool = TavilySearchResults()

In [8]:
print(tool.name)
print(type(tool))

tavily_search_results_json
<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>


In [10]:
class AgentState(TypedDict):
    messages:Annotated[list[AnyMessage],operator.add]

In [11]:
class Agent:
    def __init__(self,model,tools,system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)
        
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0
    def call_openai(self, state: AgentState):
        messages=state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages':[message]}
    def take_action(self,state:AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results=[]
        for t in tool_calls:
            print("Calling",str(t))
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}
            

In [12]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-4-turbo")
abot = Agent(model, [tool], system=prompt)

In [16]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_5l2jaGdoEBNcbBJZ6vrAfg1R'}
Back to the model!


In [17]:
result['messages'][-1].content

'The current weather in San Francisco is misty with a temperature of 46.4°F (8°C). The humidity is at 100%, and the wind is blowing from the north at 2.2 mph. Visibility is around 3 miles.'

In [18]:
messages = [HumanMessage(content="What is the weather in hyderabad, india and what is the area of apartment myhome avatar, narsingi in acres?")]
result = abot.graph.invoke({"messages": messages})

Calling {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Hyderabad, India'}, 'id': 'call_QkFq8uzhF51xbgmrrno7fWBP'}
Calling {'name': 'tavily_search_results_json', 'args': {'query': 'area of apartment My Home Avatar, Narsingi in acres'}, 'id': 'call_SK4hSTS8xznsJAmvF3MuXXHk'}
Back to the model!


In [20]:
print(result['messages'][-1].content)

### Weather in Hyderabad, India (as of June 6, 2024)
- **Temperature**: 26°C (feels like 27.6°C)
- **Weather Condition**: Moderate or heavy rain with thunder
- **Wind**: 19.1 kph from the East
- **Humidity**: 79%
- **Visibility**: 4 km

### Area of My Home Avatar, Narsingi
- **Total Area**: 22.57 acres

The weather information shows that it is currently raining in Hyderabad with moderate temperatures, while the My Home Avatar residential project spans an area of 22.57 acres, providing ample recreational and living space.
